# NANOOS SOS to WMS (SensorML > geopandas)
6/2/2016

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

from datetime import datetime, timedelta
import pytz
from urllib import urlencode

from pyoos.collectors.ioos.swe_sos import IoosSweSos
from pyoos.parsers.ioos.describe_sensor import IoosDescribeSensor

from owslib.sos import SensorObservationService
from owslib.swe.sensor.sml import SensorML, Contact
from owslib.util import (testXMLValue, testXMLAttribute, nspath_eval, 
                         xmltag_split, xml_to_dict, dict_union, extract_xml_list)
from owslib.namespaces import Namespaces

from geojson import Point, Feature, FeatureCollection
import numpy as np
import pandas as pd
import geopandas as gpd


def get_namespaces():
    n = Namespaces()
    # namespaces = n.get_namespaces(["sml", "gml", "xlink"])
    namespaces = n.get_namespaces(["sml", "gml", "xlink", "swe"])
    namespaces["ism"] = "urn:us:gov:ic:ism:v2"
    return namespaces

namespaces = get_namespaces()

def nsp(path):
    return nspath_eval(path, namespaces)

/home/lsetiawan/anaconda2/envs/uwapl_em_mc_1aui/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
# Ran with ioos conda env
%load_ext version_information
%version_information numpy, pandas, owslib, pyoos

Software versions
Python 2.7.11 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython 4.2.0
OS Linux 4.2.0 36 generic x86_64 with debian jessie sid
numpy 1.10.4
pandas 0.18.1
owslib 0.11.0
pyoos 0.7.0
Thu Jun 02 12:37:25 2016 PDT

## NANOOS SOS

In [3]:
sosurl = 'http://data.nanoos.org/52nsos/sos/kvp'

# NANOOS SOS is failing??
# ConnectionError: HTTPConnectionPool(host='data.nanoos.org', port=80): Max retries exceeded with url: 
# /52nsos/sos/kvp?service=SOS&request=GetCapabilities&acceptVersions=1.0.0 
# (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x7f34545dca50>: 
# Failed to establish a new connection: [Errno -2] Name or service not known',))

# sosurl = 'http://sos.cencoos.org/sos/sos/kvp'

In [4]:
collector52n = IoosSweSos(sosurl)
offerings52n = collector52n.server.offerings

In [5]:
# contents = collector52n.server.contents

## select station for testing

In [6]:
station_urn = 'urn:ioos:station:nanoos:apl_chaba'
# station_urn = 'urn:ioos:station:cencoos:Humboldt'

In [7]:
# procedure = station_urn
collector52n.features = [station_urn]

In [8]:
# sensorml = collector52n.metadata(feature_name_callback=procedure)
sensorml_resp_lst = collector52n.metadata(timeout=200)
sensorml_resp_lst

In [9]:
sensorml = sensorml_resp_lst[0]

In [10]:
pyoos_ds = IoosDescribeSensor(sensorml._root)
pyoos_ds

In [11]:
vars(pyoos_ds)

{'ending': datetime.datetime(2016, 6, 2, 17, 51, 16, tzinfo=tzutc()),
 'id': 'urn:ioos:station:nanoos:apl_chaba',
 'ioos_version': '1.0',
 'keywords': [],
 'location': <Element {http://www.opengis.net/gml}Point at 0x7fb1de7a7ea8>,
 'longName': u'(APL-UW) \xc4\u2020h\xc3\xa1\xca\u201dba\xc2\xb7 UW/NANOOS Moored Buoy near La Push',
 'platformType': 'moored_buoy',
 'shortName': u'APL-UW \xc4\u2020h\xc3\xa1\xca\u201dba\xc2\xb7',
 'starting': datetime.datetime(2016, 3, 19, 17, 41, 37, tzinfo=tzutc()),
 'system': <owslib.swe.sensor.sml.System at 0x7fb1de7ac0d0>,
 'variables': ['http://mmisw.org/ont/cf/parameter/air_temperature',
  'http://mmisw.org/ont/cf/parameter/mass_concentration_of_chlorophyll_in_sea_water',
  'http://mmisw.org/ont/cf/parameter/mass_concentration_of_oxygen_in_sea_water',
  'http://mmisw.org/ont/cf/parameter/mole_concentration_of_nitrate_in_sea_water',
  'http://mmisw.org/ont/cf/parameter/relative_humidity',
  'http://mmisw.org/ont/cf/parameter/sea_water_pressure',
  'ht

In [12]:
# vars(pyoos_ds.system)

In [13]:
pos = testXMLValue(pyoos_ds.system.location.find(nsp('gml:Point/gml:pos')))
pos.split()

['47.96590042114258', '-124.94920349121094']

## Code to pull out all station urn's from GetCapabilities

In [14]:
len(offerings52n), type(offerings52n), type(offerings52n[1])

(55, list, owslib.swe.observation.sos100.SosObservationOffering)

In [15]:
off1 = offerings52n[1]

In [16]:
vars(off1)

{'_root': <Element {http://www.opengis.net/sos/1.0}ObservationOffering at 0x7fb1de7e2d40>,
 'bbox': (-124.95020294189453,
  47.9659004211426,
  -124.949203491211,
  47.967201232910156),
 'bbox_srs': urn:ogc:def:crs:EPSG::4326,
 'begin_position': datetime.datetime(2014, 8, 1, 0, 0, 36, tzinfo=tzutc()),
 'description': None,
 'end_position': datetime.datetime(2016, 6, 2, 17, 51, 16, tzinfo=tzutc()),
 'features_of_interest': ['urn:ioos:station:nanoos:apl_chaba(height0.0m)'],
 'id': 'urn_ioos_station_nanoos_apl_chaba',
 'name': 'urn:ioos:station:nanoos:apl_chaba',
 'observed_properties': ['NONE',
  'http://mmisw.org/ont/cf/parameter/air_temperature',
  'http://mmisw.org/ont/cf/parameter/mass_concentration_of_chlorophyll_in_sea_water',
  'http://mmisw.org/ont/cf/parameter/mass_concentration_of_oxygen_in_sea_water',
  'http://mmisw.org/ont/cf/parameter/mole_concentration_of_nitrate_in_sea_water',
  'http://mmisw.org/ont/cf/parameter/relative_humidity',
  'http://mmisw.org/ont/cf/parameter/se

In [17]:
off1.name

'urn:ioos:station:nanoos:apl_chaba'

In [18]:
offerings52n[2].name

'urn:ioos:station:nanoos:apl_npb1ptwells'

## Harvest all stations

In [19]:
station_urn_lst = [urn.name for urn in offerings52n if urn.name.split(':')[-1] != 'all']
len(station_urn_lst)

54

In [20]:
collector52n.features = station_urn_lst
sensorml_resp_lst = collector52n.metadata(timeout=200)

In [21]:
station_rec_lst = []
for station_idx, station_urn in enumerate(station_urn_lst):
    ds = IoosDescribeSensor(sensorml_resp_lst[station_idx]._root)

    pos = testXMLValue(ds.system.location.find(nsp('gml:Point/gml:pos')))
    lat, lon = pos.split()
    station_rec = dict(
        id=ds.id,
        longitude=float(lon),
        latitude=float(lat),
        station_urn_id=ds.id,
        shortName=ds.shortName,
        longName=ds.longName,
        platformType=ds.platformType,
        starting=datetime.isoformat(ds.starting),
        ending=datetime.isoformat(ds.ending)
    )
    #     variables_lst=pyoos_ds.variables
    
    station_rec_lst.append(station_rec)

In [22]:
# station_rec
# station_rec_lst = [station_rec]

In [23]:
# geojson features and feature collection
features = []
for rec in station_rec_lst:
    feature = Feature(geometry=Point((rec['longitude'], rec['latitude'])), 
                      id=rec['id'],
                      properties=rec)
    features.append(feature)
    
stations_featcoll = FeatureCollection(features)
# stations_featcoll

## Load into geopandas

In [24]:
# Assign CRS, retrieved from epsg.io, the example below is EPSG:4326
crs = 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]'

stations_gdf = gpd.GeoDataFrame.from_features(features=stations_featcoll['features'],crs=crs)
len(stations_gdf)

54

In [25]:
stations_gdf.head()

,ending,geometry,id,latitude,longName,longitude,platformType,shortName,starting,station_urn_id
0,2016-06-02T17:51:16+00:00,POINT (-124.9492034912109 47.96590042114258),urn:ioos:station:nanoos:apl_chaba,47.965900,(APL-UW) Ä†hÃ¡Ê”baÂ· UW/NANOOS Moored Buoy nea...,-124.949203,moored_buoy,APL-UW Ä†hÃ¡Ê”baÂ·,2016-03-19T17:41:37+00:00,urn:ioos:station:nanoos:apl_chaba
1,2016-06-02T07:17:07+00:00,POINT (-122.3972015380859 47.76119995117188),urn:ioos:station:nanoos:apl_npb1ptwells,47.761200,(APL-UW) LSG NPB-1 Profiling Buoy at Pt. Wells,-122.397202,moored_buoy,APL-UW NPB-1,2015-12-04T06:28:48+00:00,urn:ioos:station:nanoos:apl_npb1ptwells
2,2016-06-02T19:08:14+00:00,POINT (-122.7300033569336 47.27999877929688),urn:ioos:station:nanoos:apl_npb2carr,47.279999,(APL-UW) LSG NPB-2 Profiling Buoy at Carr Inlet,-122.730003,moored_buoy,APL-UW NPB-2,2015-12-07T20:09:44+00:00,urn:ioos:station:nanoos:apl_npb2carr
3,2013-08-16T21:54:00+00:00,POINT (-123.7740020751953 46.2047004699707),urn:ioos:station:nanoos:cmop_coaof,46.204700,(CMOP) Waste water outfall (City of Astoria),-123.774002,fixed,CMOP_Coaof,2013-08-16T21:54:00+00:00,urn:ioos:station:nanoos:cmop_coaof
4,2015-09-07T20:02:44+00:00,POINT (-123.9551010131836 46.22570037841797),urn:ioos:station:nanoos:cmop_dsdma,46.225700,(CMOP) Desdemona Sands Light,-123.955101,fixed,CMOP_Dsdma,2014-08-01T00:14:12+00:00,urn:ioos:station:nanoos:cmop_dsdma


In [26]:
# stations_gdf.plot()

## Write Geopandas dataframe to GeoPackage

In [27]:
stations_gdf.to_file('nanoos_stations.gpkg',driver='GPKG')

In [28]:
# Check if the geopackage was written correctly
test = gpd.read_file('nanoos_stations.gpkg')
test.head()

,ending,geometry,id,latitude,longName,longitude,platformType,shortName,starting,station_urn_id
0,2016-06-02T17:51:16+00:00,POINT (-124.9492034912109 47.96590042114258),urn:ioos:station:nanoos:apl_chaba,47.965900,(APL-UW) Ä†hÃ¡Ê”baÂ· UW/NANOOS Moored Buoy nea...,-124.949203,moored_buoy,APL-UW Ä†hÃ¡Ê”baÂ·,2016-03-19T17:41:37+00:00,urn:ioos:station:nanoos:apl_chaba
1,2016-06-02T07:17:07+00:00,POINT (-122.3972015380859 47.76119995117188),urn:ioos:station:nanoos:apl_npb1ptwells,47.761200,(APL-UW) LSG NPB-1 Profiling Buoy at Pt. Wells,-122.397202,moored_buoy,APL-UW NPB-1,2015-12-04T06:28:48+00:00,urn:ioos:station:nanoos:apl_npb1ptwells
2,2016-06-02T19:08:14+00:00,POINT (-122.7300033569336 47.27999877929688),urn:ioos:station:nanoos:apl_npb2carr,47.279999,(APL-UW) LSG NPB-2 Profiling Buoy at Carr Inlet,-122.730003,moored_buoy,APL-UW NPB-2,2015-12-07T20:09:44+00:00,urn:ioos:station:nanoos:apl_npb2carr
3,2013-08-16T21:54:00+00:00,POINT (-123.7740020751953 46.2047004699707),urn:ioos:station:nanoos:cmop_coaof,46.204700,(CMOP) Waste water outfall (City of Astoria),-123.774002,fixed,CMOP_Coaof,2013-08-16T21:54:00+00:00,urn:ioos:station:nanoos:cmop_coaof
4,2015-09-07T20:02:44+00:00,POINT (-123.9551010131836 46.22570037841797),urn:ioos:station:nanoos:cmop_dsdma,46.225700,(CMOP) Desdemona Sands Light,-123.955101,fixed,CMOP_Dsdma,2014-08-01T00:14:12+00:00,urn:ioos:station:nanoos:cmop_dsdma


## Uploading Geopackage to GeoServer

In [29]:
# Import the Catalog module
from geoserver.catalog import Catalog
# Import subprocess to use cURL REST API since gsconfig, doesn't seem to have this capability anymore
import subprocess

In [30]:
# Retrieve catalog from Geoserver Instance via REST (REpresentational State Transfer)
cat = Catalog("http://data.nanoos.org/geoserver2_8/rest", username='####', password='####')

In [31]:
# Get list of workspaces
print cat.get_workspaces()

[nanoos_dev @ http://data.nanoos.org/geoserver2_8/rest/workspaces/nanoos_dev.xml, czo @ http://data.nanoos.org/geoserver2_8/rest/workspaces/czo.xml, otnnep @ http://data.nanoos.org/geoserver2_8/rest/workspaces/otnnep.xml, nvs_assets @ http://data.nanoos.org/geoserver2_8/rest/workspaces/nvs_assets.xml, xan @ http://data.nanoos.org/geoserver2_8/rest/workspaces/xan.xml, basemaps @ http://data.nanoos.org/geoserver2_8/rest/workspaces/basemaps.xml, oa @ http://data.nanoos.org/geoserver2_8/rest/workspaces/oa.xml, crb @ http://data.nanoos.org/geoserver2_8/rest/workspaces/crb.xml, partners @ http://data.nanoos.org/geoserver2_8/rest/workspaces/partners.xml, nvs @ http://data.nanoos.org/geoserver2_8/rest/workspaces/nvs.xml, drb @ http://data.nanoos.org/geoserver2_8/rest/workspaces/drb.xml]


In [32]:
# create new workspace
# cat.create_workspace('nvs_assets', 'http://data.nanoos.org/geoserver2_8/nvs_assets')

# Get workspace
nvs = cat.get_workspace('nvs_assets')
print nvs.name

nvs_assets


**Note: Upload geopackages into geoserver data directory using ssh or ftp before proceeding**
- Ex: In localhost, data is stored in ../geoserver/data/data/geopackages/


In [33]:
# Create the geopackage datastore
gpkg_ds = cat.create_datastore('nanoos_stations', workspace=nvs)
# Edit the connection parameters
gpkg_ds.connection_parameters = {'Connection timeout': '20',
 'Evictor run periodicity': '300',
 'Evictor tests per run': '3',
 'Expose primary keys': 'false',
 'Max connection idle time': '300',
 'Test while idle': 'true',
 'database': 'file:data/geopackages/nanoos_stations.gpkg', # Point to location of geopackage relative to the geoserver data directory
 'dbtype': 'geopkg',
 'fetch size': '1000',
 'max connections': '10',
 'min connections': '1',
 'namespace': 'http://data.nanoos.org/geoserver2_8/nvs_assets', # Workspace URL
 'validate connections': 'true'}
# Save datastore
cat.save(gpkg_ds)

({'connection': 'keep-alive',
  'content-length': '0',
  'date': 'Thu, 02 Jun 2016 19:37:37 GMT',
  'location': 'http://data.nanoos.org/geoserver2_8/rest/workspaces/nvs_assets/datastores/nanoos_stations',
  'server': 'nginx/1.1.19',
  'status': '201'},
 '')

In [34]:
# Set necessary variables for cURL
data_name = 'nanoos_stations'
wksp_name = nvs.name
ds_name = gpkg_ds.name
print ds_name

nanoos_stations


In [35]:
# Create layer from geopackage table
subprocess.call('curl -v -u username:password -XPOST -H "Content-type: text/xml" -d "<featureType><name>{0}</name></featureType>" http://data.nanoos.org/geoserver2_8/rest/workspaces/{1}/datastores/{2}/featuretypes'.format(data_name,wksp_name,ds_name), shell=True)

0

In [36]:
# get the newly published layer w/o any projection
layer = cat.get_layer(data_name)

# retrieve resource to assign projection
rsrc = layer.resource

# assign Layer projection
rsrc.projection = 'EPSG:4326'

# save layer
cat.save(rsrc)

({'connection': 'keep-alive',
  'content-length': '0',
  'date': 'Thu, 02 Jun 2016 19:37:41 GMT',
  'server': 'nginx/1.1.19',
  'status': '200'},
 '')

In [37]:
# Get layer style
layer.default_style.name

'point'

In [38]:
# Get all styles available in the Geoserver
all_style = cat.get_styles()
for style in all_style:
    print style.name

sitawr_watertemp_interp_wlabels
pophatch
sitawr_salinity_interp
sitawr_chlorophyll_interp
raster
generic
sitawr_barpress_css
crbsubwatersheds_lbl
capitals
sitawr_watertemp_interp
drb_subwatersheds_lbl
stats_pop.css
sitawr_watertemp_css
sitawr_oxygenconc_css
grass
poi
point
cite_lakes
crbrivers_lbl
tiger_roads
sitawr_oxygenconc_css2
line
restricted
polygon
population
crbrivers
burg
poly_landmarks
crbsubwatersheds
green
polygon_nofill
dem
simple_streams
simple_roads
giant_polygon
rain
sitawr_oxygenconc_interp
nvs_buoy


In [39]:
# Set layer default style to change the look
layer.default_style = 'nvs_buoy'
cat.save(layer)

({'connection': 'keep-alive',
  'content-length': '0',
  'date': 'Thu, 02 Jun 2016 19:37:41 GMT',
  'server': 'nginx/1.1.19',
  'status': '200'},
 '')